In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

data = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()


training_images = training_images /255.0
test_images = test_images /255.0
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)



11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2542 - accuracy: 0.9275
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1130 - accuracy: 0.9668
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0782 - accuracy: 0.9763
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0583 - accuracy: 0.9820
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0746 - accuracy: 0.9774


[0.074587881565094, 0.977400004863739]

In [ ]:
import tensorflow as tf

# Load the dataset
data = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()

# Normalize the images
training_images = training_images / 255.0
test_images = test_images / 255.0

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fit the model
model.fit(training_images, training_labels, epochs=5)

# Evaluate the model
model.evaluate(test_images, test_labels)

# Get the weights for each layer
weights = [layer.get_weights() for layer in model.layers]

# Print the weights
for i, layer_weights in enumerate(weights):
    print(f"Layer {i+1} weights:\n{layer_weights}\n")


Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2592 - accuracy: 0.9268
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1130 - accuracy: 0.9664
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0787 - accuracy: 0.9765
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0584 - accuracy: 0.9822
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0723 - accuracy: 0.9785
Layer 1 weights:
[]

Layer 2 weights:
[array([[ 0.07067696, -0.06764536,  0.01777177, ...,  0.00804672,
         0.08106031,  0.02961928],
       [-0.01045021, -0.07152922,  0.03830954, ...,  0.06530561,
         0.08012142, -0.02478861],
       [-0.03126684,  0.07459279, -0.03694484, ...,  0.03221045,
        -0.03161872, -0.05312537],
       ...,
       [ 0.03426677,  0.07626653,  0.04966601, ..., -0.03212664,
        -0.04109431,  0.02738893],
       [-0.07950973, -0.02391304,  0.0480222 ,

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess the dataset
data = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

# Define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Custom callback to track unchanged weights
class WeightsTracker(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        # Save the initial weights
        self.previous_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]
        self.unchanged_weights = [[np.zeros_like(w) for w in layer_weights] for layer_weights in self.previous_weights]

    def on_epoch_end(self, epoch, logs=None):
        current_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]
        print(f"\nWeights comparison after epoch {epoch + 1}:")

        # Compare current weights with previous weights and track unchanged weights
        for layer_idx, (prev_w, curr_w) in enumerate(zip(self.previous_weights, current_weights)):
            print(f"Layer {layer_idx}:")
            for w_idx, (prev, curr) in enumerate(zip(prev_w, curr_w)):
                unchanged_mask = prev == curr  # Create a mask of unchanged weights
                unchanged_count = np.sum(unchanged_mask)
                total_count = unchanged_mask.size

                # Track unchanged weights for each sub-layer
                self.unchanged_weights[layer_idx][w_idx] = unchanged_mask

                print(f"  Sub-layer {w_idx} - Unchanged Weights: {unchanged_count}/{total_count}")

        # Update previous weights to current for the next epoch comparison
        self.previous_weights = current_weights

    def on_train_end(self, logs=None):
        print("\nFinal unchanged weights mask at the end of training:")
        for layer_idx, weights in enumerate(self.unchanged_weights):
            print(f"Layer {layer_idx}:")
            for w_idx, mask in enumerate(weights):
                unchanged_count = np.sum(mask)
                total_count = mask.size
                print(f"  Sub-layer {w_idx} - Unchanged Weights: {unchanged_count}/{total_count}")

# Instantiate the callback
weights_tracker = WeightsTracker()

# Train the model with the custom callback
model.fit(training_images, training_labels, epochs=10, callbacks=[weights_tracker])

# Evaluate the model
model.evaluate(test_images, test_labels)



11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1863/1875 [============================>.] - ETA: 0s - loss: 0.2636 - accuracy: 0.9250
Weights comparison after epoch 1:
Layer 0:
  Sub-layer 0 - Unchanged Weights: 11473/100352
  Sub-layer 1 - Unchanged Weights: 0/128
Layer 1:
  Sub-layer 0 - Unchanged Weights: 0/1280
  Sub-layer 1 - Unchanged Weights: 0/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2629 - accuracy: 0.9252
Epoch 2/10
1855/1875 [============================>.] - ETA: 0s - loss: 0.1144 - accuracy: 0.9667
Weights comparison after epoch 2:
Layer 0:
  Sub-layer 0 - Unchanged Weights: 13546/100352
  Sub-layer 1 - Unchanged Weights: 1/128
Layer 1:
  Sub-layer 0 - Unchanged Weights: 10/1280
  Sub-layer 1 - Unchanged Weights: 0/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1138 - accuracy: 0.9668
Epoch 3/10
1864/1875 [============================>.] - ETA: 0s - loss: 0.0788 - accuracy: 0.9763
Weights comp

[0.08351504802703857, 0.9757999777793884]

#**new**

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load and preprocess data
data = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

# Define the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])



In [17]:
# Custom callback to track unchanged weights
class WeightChangeTracker(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        # Save initial weights
        self.initial_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]

    def on_epoch_end(self, epoch, logs=None):
        unchanged_weights = []
        current_weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]
        print('\n')

        for i, (initial_layer_weights, current_layer_weights) in enumerate(zip(self.initial_weights, current_weights)):
            unchanged = 0
            for w_init, w_curr in zip(initial_layer_weights, current_layer_weights):
                unchanged += np.sum(w_init == w_curr)  # Count unchanged weights

            unchanged_weights.append(unchanged)
            print(f'Epoch {epoch + 1}, Layer {i + 1}: {unchanged} unchanged weights')

        # Update the initial weights for the next epoch
        self.initial_weights = current_weights

# Fit the model with the custom callback
model.fit(training_images, training_labels, epochs=10, callbacks=[WeightChangeTracker()])

# Evaluate the model
model.evaluate(test_images, test_labels)


Epoch 1/10
1856/1875 [============================>.] - ETA: 0s - loss: 0.0071 - accuracy: 0.9979

Epoch 1, Layer 1: 14538 unchanged weights
Epoch 1, Layer 2: 10 unchanged weights
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0071 - accuracy: 0.9979
Epoch 2/10
1865/1875 [============================>.] - ETA: 0s - loss: 0.0069 - accuracy: 0.9977

Epoch 2, Layer 1: 14478 unchanged weights
Epoch 2, Layer 2: 10 unchanged weights
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0069 - accuracy: 0.9977
Epoch 3/10
1863/1875 [============================>.] - ETA: 0s - loss: 0.0071 - accuracy: 0.9979

Epoch 3, Layer 1: 14589 unchanged weights
Epoch 3, Layer 2: 10 unchanged weights
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0071 - accuracy: 0.9979
Epoch 4/10
1860/1875 [============================>.] - ETA: 0s - loss: 0.0057 - accuracy: 0.9984

Epoch 4, Layer 1: 14746 unchanged weights
Epoch 4, Layer 2: 10 unchanged weights
1875/18

[0.11021310836076736, 0.9796000123023987]